In [104]:
import time
import requests
import numpy as np
import pandas as pd
import fitparse as fp
from itertools import islice
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output

#### User inputs/parameters
> Ensure you've downloaded all modules imported above. Define the parameters below, then run all cells. Visualization will run in the last cell.

In [118]:
garmin_fit_file = './5222879530.fit'
google_maps_API_key = ''
viz_title = 'My Hike'

# if your Garmin GPS data is off, you can adjust it by inputing your own latitude and longitute coordinates
#     and elevation (meters) for the summit. Summit coordinates are often available online. If you're GPS was
#     properly calibrated, leave as False

summit_lat = 36.52330
summit_long = 118.238
summit_elev = 4277

#### Import .fit file

In [106]:
fitfile = fp.FitFile(garmin_fit_file)

#### Parse fit data, convert to dictionary, then to DataFrame

In [107]:
fitfile.parse()

dict_list = []
for record in fitfile.get_messages('record'):
    dict_list.append(record.get_values())

fit_dict = {k: [d[k] for d in dict_list] for k in dict_list[0]}

hike_df = pd.DataFrame(fit_dict)
hike_df = hike_df.dropna()
hike_df.head()

,timestamp,position_lat,position_long,distance,enhanced_speed,enhanced_altitude,unknown_87,unknown_88,heart_rate,cadence,temperature,fractional_cadence
12,2020-07-11 13:32:58,434906385.0,-1.409830e+09,69.58,1.605,2898.4,0,100,122,56,23,0.5
13,2020-07-11 13:32:59,434906139.0,-1.409831e+09,71.14,1.605,2902.4,0,100,121,56,23,0.0
14,2020-07-11 13:33:00,434906139.0,-1.409831e+09,72.73,1.596,2900.6,0,100,121,56,23,0.0
15,2020-07-11 13:33:02,434905718.0,-1.409832e+09,74.28,1.568,2899.2,0,100,120,56,23,0.0
16,2020-07-11 13:33:07,434905499.0,-1.409833e+09,81.37,1.493,2899.2,0,100,120,54,23,0.0


#### Sample 4 percent of data from necessary columns, scale data

In [108]:
sampled_df = hike_df.iloc[::25, [0,1, 2,3, 5, 8]]
sampled_df['position_lat'] = sampled_df['position_lat'] / 10000000
sampled_df['position_long'] = sampled_df['position_long'] / 10000000
sampled_df['distance'] = sampled_df['distance'] / 1000
sampled_df = sampled_df.reset_index(drop=True)


#### Create total altitude gain column

In [109]:
gain = sampled_df['enhanced_altitude'] - sampled_df['enhanced_altitude'].shift(+1)
gain = gain.fillna(0)
gain = gain.map(lambda x: 0 if x < 0 else x)

sampled_df['elev_gain'] = gain.cumsum()

#### Adjust lat and long coords if summit_lat and summit_long are defined

In [110]:
if summit_lat and summit_long and summit_elev:
    lat_diff = sampled_df.loc[summit_idx, 'position_lat'] - summit_lat 
    long_diff = sampled_df.loc[summit_idx, 'position_long'] - summit_long
    elev_diff = summit_elev - sampled_df.loc[summit_idx, 'enhanced_altitude']
    sampled_df['position_lat'] = sampled_df['position_lat'] - lat_diff
    sampled_df['position_long'] = sampled_df['position_long'] - long_diff
    sampled_df['enhanced_altitude'] = sampled_df['enhanced_altitude'] + elev_diff

#### Set topo boundaries to hike boundaries

In [111]:
s_most = sampled_df['position_lat'].min()
n_most = sampled_df['position_lat'].max()
w_most = sampled_df['position_long'].min()
e_most = sampled_df['position_long'].max()

#### Create lat and long arrays of points between outermost points

In [112]:
lat_coords = np.linspace(s_most, n_most, 50)
long_coords = np.linspace(w_most, e_most, 50)

#### Convert lat_coords and long_coords to list of string x, y pairs

In [113]:
coords = [(str(i) + ',' + str(j)) for i in lat_coords for j in long_coords]

#### Call Maps Elevation API for each point in coords, reshape X, Y, Z

In [114]:
base_url = 'https://maps.googleapis.com/maps/api/elevation/json?locations='
key = f'&key={google_maps_API_key}'

Z = []
for point in coords:
    res = requests.get(base_url + point + key)
    data = res.json()
    Z.append(data['results'][0]['elevation'])


#### Set hike path to X, Y, Z numpy arrays

In [115]:
X_path = np.array(sampled_df['position_long'])
Y_path = np.array(sampled_df['position_lat'])
Z_path = np.array(sampled_df['enhanced_altitude'])

In [ ]:
Z = np.reshape(np.array(Z), (50,50))
X, Y = np.meshgrid(long_coords, lat_coords)

In [116]:
print(X_path[:10], '\n')
print(X[0][:10], '\n\n')
print(Y_path[:10], '\n')
print(Y[0][:10], '\n\n')
print(Z_path[:10], '\n')
print(Z[0][:10], '\n\n')


[118.3201325 118.3184488 118.3155435 118.3128929 118.3111673 118.3115153
 118.3114362 118.3110994 118.3098814 118.3085291] 

[118.2313311  118.23316287 118.23499463 118.2368264  118.23865817
 118.24048994 118.2423217  118.24415347 118.24598524 118.24781701] 


[36.4399014 36.4398866 36.4399904 36.440398  36.4417829 36.4431724
 36.4445549 36.4460225 36.4476095 36.4483087] 

[36.4381836 36.4381836 36.4381836 36.4381836 36.4381836 36.4381836
 36.4381836 36.4381836 36.4381836 36.4381836] 


[3130.4 3140.8 3147.4 3154.8 3155.8 3148.8 3143.6 3142.4 3142.2 3136. ] 

[698.47998047 650.98504639 591.4744873  585.58721924 640.46337891
 707.52093506 743.14892578 732.71203613 686.66491699 642.03887939] 




#### Create plot

In [119]:
a = np.linspace(0, 360, 361)
b = np.array([a,a,a,a,a,a])
b = np.ravel(b)

for x in range(0, len(X_path), 2):
    
    clear_output(wait=True)
    fig = plt.figure(figsize=(14,10), constrained_layout=True)
    ax = plt.axes(projection='3d')
    ax.view_init(50, b[x])
    
    title_string = "%s\n\nDistance: %.2f km\nElevation Gain: %i m\nTime (PST): %s" % (viz_title, round(sampled_df.loc[x, 'distance'],2), sampled_df.loc[x, 'elev_gain'], sampled_df.loc[x, 'timestamp'].strftime("%H:%M"))
    ax.set_title(title_string, fontsize=24, loc='left')
    ax.plot_wireframe(X, Y, Z, color='k', alpha = 0.5)
    ax.scatter(X_path[:x+2], Y_path[:x+2], Z_path[:x+2], s=50,
               vmin = 100, vmax = 160, c=sampled_df.loc[:x+1, 'heart_rate'], cmap=plt.cm.cool)
    
    sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(vmin=100, vmax=160))
    sm.set_array([])
    clb = fig.colorbar(sm, shrink=0.4, ax=ax, location='left')
    clb.ax.set_title('Heart Rate\n')
    
    plt.axis('off')
    plt.show()

KeyboardInterrupt: 